# ISTAC playground

**NOTE: You must select the istac kernel to run this playbook!**

This playbook uses a virtualenv that must be setup beforehand. This is a one-time only process, recorded here for future reference.
The steps to create a virtualenv for this project and make it available as a jupyter kernel are:

```bash
# Move to the proper folder
cd /home/jovyan/work/istac
# Install pipenv
pip install pipenv
# Create the venv and install dependencies
pipenv install
# Activate the shell
pipenv shell
# Create a jupyter core
python -m ipykernel install --user --name=istac
```

Now you can select the "istac" core when running this playbook.

In [1]:
# Import istac lib into your app
import istac

In [2]:
# Collect all indicators
import aiohttp

async with aiohttp.ClientSession() as session:
    indicators = [ind async for ind in istac.indicators(session)]

In [3]:
# Now you can list the indicators, e.g.
from pprint import pprint

pprint([ind.code for ind in indicators[:10]])

['AFILIACIONES',
 'TURISTAS',
 'EMPLEO_REGISTRADO_AGRICULTURA',
 'EMPLEO_REGISTRADO_HOSTELERIA',
 'EMPLEO_REGISTRADO_INDUSTRIA',
 'EMPLEO_REGISTRADO_SERVICIOS',
 'POBLACION_INACTIVA',
 'POBLACION_INACTIVA_HOMBRES',
 'POBLACION_INACTIVA_MUJERES',
 'PARO_REGISTRADO']


In [4]:
# And optionally, turn the list into a dataframe
import pandas as pd

ind_frame = pd.DataFrame(ind.attribs for ind in indicators).set_index('id')
for col in ('selfLink', 'systemSurveyLinks', 'kind'):
    ind_frame = ind_frame.drop(col, axis=1)
ind_frame.head()

,code,version,title,subjectCode,subjectTitle,conceptDescription,notes
id,,,,,,,
AFILIACIONES,AFILIACIONES,1.13,"{'es': 'Afiliaciones a la Seguridad Social', '...",051,"{'es': '051 Empleo', '__default__': '051 Empleo'}",{'es': 'Puestos de trabajo registrados en la S...,{'en': 'Affiliations registered on data collec...
TURISTAS,TURISTAS,1.19,"{'es': 'Turistas recibidos', 'en': 'Tourists a...",082,"{'es': '082 Hostelería y turismo', '__default_...",{'es': 'Número de turistas recibidos por vía a...,{'en': 'Tourists are visitors who overnight in...
EMPLEO_REGISTRADO_AGRICULTURA,EMPLEO_REGISTRADO_AGRICULTURA,1.23,"{'en': 'Registered employment. Agriculture', '...",051,"{'es': '051 Empleo', '__default__': '051 Empleo'}","{'en': 'Jobs registered in the primary sector,...",{'es': 'En el sector primario se contabiliza c...
EMPLEO_REGISTRADO_HOSTELERIA,EMPLEO_REGISTRADO_HOSTELERIA,1.23,"{'es': 'Empleo registrado. Hostelería', 'en': ...",051,"{'es': '051 Empleo', '__default__': '051 Empleo'}",{'es': 'Puestos de trabajo registrados en la s...,{'es': 'Se entiende por empleo registrado a la...
EMPLEO_REGISTRADO_INDUSTRIA,EMPLEO_REGISTRADO_INDUSTRIA,1.24,"{'es': 'Empleo registrado. Industria', 'en': '...",051,"{'es': '051 Empleo', '__default__': '051 Empleo'}",{'en': 'Jobs registered in the industry and en...,{'es': 'Se entiende por empleo registrado a la...


In [5]:
# Let's save the list
ind_frame.to_csv(r'indicadores.csv', index=None, header=True)

In [6]:
import asyncio

# Get data (with dimensions) for some indicator
async def get_data(indicator):
    # Collect data and dimensions in parallel
    async with aiohttp.ClientSession() as session:
        df, dims = await asyncio.gather(
            # Get indicator data
            istac.indicator_df(session, indicator, {
                #'granularity': 'TIME[MONTHLY]',
                #'representation': 'MEASURE[ABSOLUTE]',
                'fields': '-observationsMetadata'
            }),
            # Get dimensions for this same indicator
            istac.dimensions(session, indicator)
        )
    # Join the data with the dimensions
    return istac.Indicator.join(df, dims, dropna=True)

## Get indicator data

In [7]:
import ipywidgets as widgets

options = sorted([ind.code for ind in indicators])
selection = widgets.Dropdown(
    options=options,
    value=options[0],
    description='Indicador:',
    disabled=False,
)

selection

Dropdown(description='Indicador:', options=('ACCIDENTES_TRABAJO_BAJA', 'ACCIDENTES_TRABAJO_BAJA_JORNADAS', 'AF…

In [9]:
# Export the indicator to CSV
indicador = selection.value
data = await get_data(indicador)
data.to_csv(f'{indicador}.csv', index=True, header=True)
pprint(data.head().to_csv())

('GEOGRAPHICAL,TIME,MEASURE,F,title,granularityCode,latitude,longitude,title_TIME,granularityCode_TIME,title_MEASURE,decimalPlaces,isPercentage,type,unit,unitMultiplier,unitSymbol,unitSymbolPosition\n'
 'ES70,2019Q4,ANNUAL_PERCENTAGE_RATE,1.48,Canarias,REGIONS,28.2869925,-15.8335245,2019 '
 'Cuarto trimestre,QUARTERLY,Tasa variación '
 'anual,2,True,CHANGE_RATE,Porcentaje,,%,END\n'
 'ES70,2019Q4,INTERPERIOD_PERCENTAGE_RATE,1.71,Canarias,REGIONS,28.2869925,-15.8335245,2019 '
 'Cuarto trimestre,QUARTERLY,Tasa variación '
 'interperiódica,2,True,CHANGE_RATE,Porcentaje,,%,END\n'
 'ES70,2019Q4,ABSOLUTE,824869,Canarias,REGIONS,28.2869925,-15.8335245,2019 '
 'Cuarto trimestre,QUARTERLY,Dato,0,,AMOUNT,Puestos de trabajo,Unidades,,\n'
 'ES70,2019Q4,ANNUAL_PUNTUAL_RATE,12024,Canarias,REGIONS,28.2869925,-15.8335245,2019 '
 'Cuarto trimestre,QUARTERLY,Variación anual,0,,AMOUNT,Puestos de '
 'trabajo,Unidades,,\n'
 'ES70,2019Q4,INTERPERIOD_PUNTUAL_RATE,13904,Canarias,REGIONS,28.2869925,-15.8335245,